In [1]:
import warnings
warnings.filterwarnings('ignore')
import stwcs
import glob
import sys
import os
import shutil
import time
import filecmp
import astroquery
import progressbar
import copy
import requests
import random
import astropy.wcs as wcs
import numpy as np
from contextlib import contextmanager
from astropy import units as u
from astropy.utils.data import clear_download_cache,download_file
from astropy.io import fits
from astropy.table import Table, Column, unique
from astropy.time import Time
from astroscrappy import detect_cosmics
from stwcs import updatewcs
from scipy.interpolate import interp1d

# Internal dependencies
from common import Constants
from common import Options
from common import Settings
from common import Util
from nbutils import get_filter, get_instrument, get_chip, get_filter
from nbutils import get_zpt, add_visit_info, organize_reduction_tables, pick_deepest_images

@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

with suppress_stdout():
    from drizzlepac import tweakreg,astrodrizzle,catalogs,photeq
    from astroquery.mast import Observations
    from astropy.coordinates import SkyCoord

In [2]:
acceptable_filters = [
    'F220W','F250W','F330W','F344N','F435W','F475W','F550M','F555W',
    'F606W','F625W','F660N','F660N','F775W','F814W','F850LP','F892N',
    'F098M','F105W','F110W','F125W','F126N','F127M','F128N','F130N','F132N',
    'F139M','F140W','F153M','F160W','F164N','F167N','F200LP','F218W','F225W',
    'F275W','F280N','F300X','F336W','F343N','F350LP','F373N','F390M','F390W',
    'F395N','F410M','F438W','F467M','F469N','F475X','F487N','F547M',
    'F600LP','F621M','F625W','F631N','F645N','F656N','F657N','F658N','F665N',
    'F673N','F680N','F689M','F763M','F845M','F953N','F122M','F160BW','F185W',
    'F218W','F255W','F300W','F375N','F380W','F390N','F437N','F439W','F450W',
    'F569W','F588N','F622W','F631N','F673N','F675W','F702W','F785LP','F791W',
    'F953N','F1042M']

### Get obstable

In [3]:
workdir = '.'
input_images = [s for p in ['*flc.fits', '*flt.fits'] for s in glob.glob(os.path.join(workdir, p))]

In [4]:
img = input_images
zptype = 'abmag'
good = []
image_number = []
#check if image exists (check in archive otherwise)
for image in img:
    success = True
    if not os.path.exists(image):
        success = False
    if success:
        good.append(image)
img = copy.copy(good)

hdu = fits.open(img[0])
h = hdu[0].header

exp = [fits.getval(image,'EXPTIME') for image in img] #exposure time
if 'DATE-OBS' in h.keys() and 'TIME-OBS' in h.keys(): 
    dat = [fits.getval(image,'DATE-OBS') + 'T' +
           fits.getval(image,'TIME-OBS') for image in img] #datetime
elif 'EXPSTART' in h.keys():
    dat = [Time(fits.getval(image, 'EXPSTART'),
        format='mjd').datetime.strftime('%Y-%m-%dT%H:%M:%S') #datetime if DATE-OBS is missing
        for image in img]
    
fil = [get_filter(image) for image in img]
ins = [get_instrument(image) for image in img]
det = ['_'.join(get_instrument(image).split('_')[:2]) for image in img]
chip= [get_chip(image) for image in img]
zpt = [get_zpt(i, ccdchip=c, zptype=zptype) for i,c in zip(img,chip)]

if not image_number:
    image_number = [0 for image in img]
    
obstable = Table([img,exp,dat,fil,ins,det,zpt,chip,image_number],
    names=['image','exptime','datetime','filter','instrument',
     'detector','zeropoint','chip','imagenumber'])
obstable.sort('datetime')
obstable = add_visit_info(obstable)

In [5]:
obstable.add_column(Column([' '*99]*len(obstable), name='drizname'))
for i,row in enumerate(obstable):
    visit = row['visit']
    n = str(visit).zfill(4)
    inst = row['instrument']
    filt = row['filter']

    # Visit should correspond to first image so they're all the same
    visittable = obstable[obstable['visit']==visit]
    refimage = visittable['image'][0]
    if 'DATE-OBS' in h.keys():
        date_obj = Time(fits.getval(refimage, 'DATE-OBS'))
    else:
        date_obj = Time(fits.getval(refimage, 'EXPSTART'), format='mjd')
    date_str = date_obj.datetime.strftime('%y%m%d')

    # Make a photpipe-like image name
    drizname = ''
    objname = None
    if objname:
        drizname = '{obj}.{inst}.{filt}.ut{date}_{n}.drz.fits'
        drizname = drizname.format(inst=inst.split('_')[0],
            filt=filt, n=n, date=date_str, obj=objname)
    else:
        drizname = '{inst}.{filt}.ut{date}_{n}.drz.fits'
        drizname = drizname.format(inst=inst.split('_')[0],
            filt=filt, n=n, date=date_str)

    if '.':
        drizname = os.path.join('.', drizname)

    obstable[i]['drizname'] = drizname

In [6]:
obstable = obstable[:12]

In [7]:
obstable

image,exptime,datetime,filter,instrument,detector,zeropoint,chip,imagenumber,visit,drizname
str20,float64,str19,str5,str14,str9,float64,str21,int64,int64,str99
./jbqz22siq_flc.fits,480.0,2012-01-04T08:08:04,f606w,acs_wfc_full,acs_wfc,26.491377676285538,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz22skq_flc.fits,480.0,2012-01-04T08:18:42,f606w,acs_wfc_full,acs_wfc,26.491377676285538,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz22smq_flc.fits,480.0,2012-01-04T08:29:20,f606w,acs_wfc_full,acs_wfc,26.491377676285538,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz22soq_flc.fits,480.0,2012-01-04T08:39:58,f606w,acs_wfc_full,acs_wfc,26.491377676285538,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz23tqq_flc.fits,480.0,2012-01-04T16:06:48,f606w,acs_wfc_full,acs_wfc,26.491377137444424,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz23trq_flc.fits,480.0,2012-01-04T16:17:26,f606w,acs_wfc_full,acs_wfc,26.491377137444424,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz23ttq_flc.fits,480.0,2012-01-04T16:28:04,f606w,acs_wfc_full,acs_wfc,26.491377137444424,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz23tvq_flc.fits,480.0,2012-01-04T16:38:42,f606w,acs_wfc_full,acs_wfc,26.491377054545822,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz24tyq_flc.fits,480.0,2012-01-04T18:08:11,f606w,acs_wfc_full,acs_wfc,26.491376957830788,1,0,1,./acs.f606w.ut120104_0001.drz.fits


### Tweakreg

In [20]:
def tweakreg_error(exception):
    message = '\n\n' + '#'*80 + '\n'
    message += 'WARNING: tweakreg failed: {e}\n'
    message += '#'*80 + '\n'
    print(message.format(e=exception.__class__.__name__))
    print('Error:', exception)
    print('Adjusting thresholds and images...')

In [21]:
# Apply TWEAKSUC header variable if tweakreg was successful
def apply_tweakreg_success(shifts):

    for row in shifts:
        if ~np.isnan(row['xoffset']) and ~np.isnan(row['yoffset']):
            file=row['file']
            if not os.path.exists(file):
                file=row['file']
                print(f'WARNING: {file} does not exist!')
                continue
            hdu = fits.open(file, mode='update')
            hdu[0].header['TWEAKSUC']=1
            hdu.close()

In [30]:
def fix_hdu_wcs_keys(image, change_keys, ref_url):

    hdu = fits.open(image, mode='update')
    ref = ref_url.strip('.old')
    outdir = '.'
    if not outdir:
        outdir = '.'

    for i,h in enumerate(hdu):
        for key in hdu[i].header.keys():
            if 'WCSNAME' in key:
                print('WCSNAME', hdu[i].header[key])
                hdu[i].header[key] = hdu[i].header[key].strip()
        for key in change_keys:
            if key in list(hdu[i].header.keys()):
                val = hdu[i].header[key]
                print(key, val)
            else:
                continue
            if val == 'N/A':
                continue
            if (ref+'$' in val):
                ref_file = val.split('$')[1]
            else:
                ref_file = val

            fullfile = os.path.join(outdir, ref_file)
            if not os.path.exists(fullfile):
                print(f'Grabbing: {fullfile}')
                # Try using both old cdbs database and new crds link
                urls = []
                url = 'https://hst-crds.stsci.edu/unchecked_get/references/hst/'
                urls.append(url+ref_file)

                url = 'ftp://ftp.stsci.edu/cdbs/'
                urls.append(url+ref_url+'/'+ref_file)

                for url in urls:
                    message = f'Downloading file: {url}'
                    sys.stdout.write(message)
                    sys.stdout.flush()
                    try:
                        dat = download_file(url, cache=False,
                            show_progress=False, timeout=120)
                        shutil.move(dat, fullfile)
                        message = '\r' + message
                        message += Constants.green+' [SUCCESS]'+Constants.end+'\n'
                        sys.stdout.write(message)
                        break
                    except:
                        message = '\r' + message
                        message += Constants.red+' [FAILURE]'+Constants.end+'\n'
                        sys.stdout.write(message)
                        print(message)

            message = f'Setting {image},{i} {key}={fullfile}'
            print(message)
            hdu[i].header[key] = fullfile

        # WFPC2 does not have residual distortion corrections and astrodrizzle
        # choke if DGEOFILE is in header but not NPOLFILE.  So do a final check
        # for this part of the WCS keys
        if 'wfpc2' in get_instrument(image).lower():
            keys = list(h.header.keys())
            if 'DGEOFILE' in keys and 'NPOLFILE' not in keys:
                del hdu[i].header['DGEOFILE']

    hdu.writeto(image, overwrite=True, output_verify='silentfix')
    hdu.close()

In [33]:
  def fix_idcscale(image):

    det = '_'.join(self.get_instrument(image).split('_')[:2])

    if 'wfc3' in det:
        hdu = fits.open(image)
        idcscale = 0.1282500028610229
        for i,h in enumerate(hdu):
            if 'IDCSCALE' not in hdu[i].header.keys():
                hdu[i].header['IDCSCALE']=idcscale

        hdu.writeto(image, overwrite=True, output_verify='silentfix')

In [34]:
# Update image wcs using updatewcs routine
def update_image_wcs(image, use_db=True):

    hdu = fits.open(image, mode='readonly')
    # Check if tweakreg was successfully run.  If so, then skip
    if 'TWEAKSUC' in hdu[0].header.keys() and hdu[0].header['TWEAKSUC']==1:
        return(True)

    # Check for hierarchical alignment.  If image has been shifted with
    # hierarchical alignment, we don't want to shift it again
    if 'HIERARCH' in hdu[0].header.keys() and hdu[0].header['HIERARCH']==1:
        return(True)

    hdu.close()

    message = 'Updating WCS for {file}'
    print(message.format(file=image))

#     self.clear_downloads(self.options['global_defaults'])

    change_keys = ['IDCTAB','DGEOFILE','NPOLEXT','NPOLFILE','D2IMFILE', 'D2IMEXT','OFFTAB']
    inst = get_instrument(image).split('_')[0]
    ref_url = 'jref.old'

    fix_hdu_wcs_keys(image, change_keys, ref_url)

    # Usually if updatewcs fails, that means it's already been done
    try:
        updatewcs.updatewcs(image, use_db=use_db) #probably not needed for jwst
        hdu = fits.open(image, mode='update')
        message = '\n\nupdatewcs success.  File info:'
        print(message)
        hdu.info()
        hdu.close()
        self.fix_hdu_wcs_keys(image, change_keys, ref_url)
        self.fix_idcscale(image)
        return(True)
    except:
        error = 'ERROR: failed to update WCS for image {file}'
        print(error.format(file=image))
        return(None)

In [40]:
def run_cosmic(image, options, output=None):
    message = 'Cleaning cosmic rays in image: {image}'
    print(message.format(image=image))
    hdulist = fits.open(image,mode='readonly')

    if output is None:
        output = image

    for i,hdu in enumerate(hdulist):
        if hdu.name=='SCI':
            mask = np.zeros(hdu.data.shape, dtype=np.bool_)

            crmask, crclean = detect_cosmics(hdu.data.copy().astype('<f4'),
                inmask=mask, readnoise=options['rdnoise'], gain=options['gain'],
                satlevel=options['saturate'], sigclip=options['sig_clip'],
                sigfrac=options['sig_frac'], objlim=options['obj_lim'])

            hdulist[i].data[:,:] = crclean[:,:]

            # Add crmask data to DQ array or DQ image
            if False:
                if 'flc' in image or 'flt' in image:
                    # Assume this hdu is corresponding DQ array
                    if len(hdulist)>=i+2 and hdulist[i+2].name=='DQ':
                        hdulist[i+2].data[np.where(crmask)]=4096
                elif 'c0m' in image:
                    maskfile = image.split('_')[0]+'_c1m.fits'
                    if os.path.exists(maskfile):
                        maskhdu = fits.open(maskfile)
                        maskhdu[i].data[np.where(crmask)]=4096
                        maskhdu.writeto(maskfile, overwrite=True)

    # This writes in place
    hdulist.writeto(output, overwrite=True, output_verify='silentfix')
    hdulist.close()

In [23]:
outdir = '.'
reference = ''
run_images = list(obstable['image'])
shift_table = Table([run_images, [np.nan]*len(run_images), 
                     [np.nan]*len(run_images)], names = ('file', 'xoffset', 'yoffset'))
tmp_images = []

In [36]:
for image in run_images:
    if True and not False:
        det = '_'.join(get_instrument(image).split('_')[:2])
        update_image_wcs(image)

Updating WCS for ./jbqz22siq_flc.fits
IDCTAB jref$4bb1536cj_idc.fits
Grabbing: ./4bb1536cj_idc.fits
Setting ./jbqz22siq_flc.fits,0 IDCTAB=./4bb1536cj_idc.fits
DGEOFILE jref$qbu16424j_dxy.fits
Grabbing: ./qbu16424j_dxy.fits
Setting ./jbqz22siq_flc.fits,0 DGEOFILE=./qbu16424j_dxy.fits
NPOLFILE jref$4bb1536ej_npl.fits
Grabbing: ./4bb1536ej_npl.fits
Setting ./jbqz22siq_flc.fits,0 NPOLFILE=./4bb1536ej_npl.fits
D2IMFILE jref$4bb15371j_d2i.fits
Grabbing: ./4bb15371j_d2i.fits
Setting ./jbqz22siq_flc.fits,0 D2IMFILE=./4bb15371j_d2i.fits
WCSNAME OPUS
WCSNAME IDC_4bb1536cj-FIT_REL_GAIAeDR3
WCSNAME IDC_4bb1536cj
WCSNAME IDC_4bb1536cj-GSC240-1
IDCTAB jref$4bb1536cj_idc.fits
Setting ./jbqz22siq_flc.fits,1 IDCTAB=./4bb1536cj_idc.fits
NPOLEXT jref$4bb1536ej_npl.fits
Setting ./jbqz22siq_flc.fits,1 NPOLEXT=./4bb1536ej_npl.fits
D2IMEXT jref$4bb15371j_d2i.fits
Setting ./jbqz22siq_flc.fits,1 D2IMEXT=./4bb15371j_d2i.fits
WCSNAME IDC_4bb1536cj
WCSNAME IDC_4bb1536cj
WCSNAME OPUS
WCSNAME IDC_4bb1536cj-FIT_REL_

Processing /home/aswin/hst123/jbqz22siq_flc.fits['SCI',1]
Updating header for /home/aswin/hst123/jbqz22siq_flc.fits[1]
Processing /home/aswin/hst123/jbqz22siq_flc.fits['SCI',2]
Updating header for /home/aswin/hst123/jbqz22siq_flc.fits[4]
Archived IDC_4bb1536cj-GSC240 in ('SCI', 1)
Archived IDC_4bb1536cj-GSC240 in ('SCI', 2)
Writing out a priori WCS IDC_4bb1536cj-GSC240 to headerlet file: jbqz22siq_flc_9ec245_hlet.fits




updatewcs success.  File info:
Filename: ./jbqz22siq_flc.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     285   ()      
  1  SCI           1 ImageHDU       263   (4096, 2048)   float32   
  2  ERR           1 ImageHDU        55   (4096, 2048)   float32   
  3  DQ            1 ImageHDU        47   (4096, 2048)   int16   
  4  SCI           2 ImageHDU       261   (4096, 2048)   float32   
  5  ERR           2 ImageHDU        55   (4096, 2048)   float32   
  6  DQ            2 ImageHDU        47   (4096, 2048)   int16   
  7  D2IMARR       1 ImageHDU        16   (64, 32)   float32   
  8  D2IMARR       2 ImageHDU        16   (64, 32)   float32   
  9  D2IMARR       3 ImageHDU        16   (64, 32)   float32   
 10  D2IMARR       4 ImageHDU        16   (64, 32)   float32   
 11  WCSDVARR      1 ImageHDU        16   (64, 32)   float32   
 12  WCSDVARR      2 ImageHDU        16   (64, 32)   float32   
 13  WCSDVARR      3 ImageHDU      

Processing /home/aswin/hst123/jbqz22skq_flc.fits['SCI',1]
Updating header for /home/aswin/hst123/jbqz22skq_flc.fits[1]
Processing /home/aswin/hst123/jbqz22skq_flc.fits['SCI',2]
Updating header for /home/aswin/hst123/jbqz22skq_flc.fits[4]
Archived IDC_4bb1536cj-GSC240 in ('SCI', 1)
Archived IDC_4bb1536cj-GSC240 in ('SCI', 2)
Writing out a priori WCS IDC_4bb1536cj-GSC240 to headerlet file: jbqz22skq_flc_9ec245_hlet.fits




updatewcs success.  File info:
Filename: ./jbqz22skq_flc.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     285   ()      
  1  SCI           1 ImageHDU       263   (4096, 2048)   float32   
  2  ERR           1 ImageHDU        55   (4096, 2048)   float32   
  3  DQ            1 ImageHDU        47   (4096, 2048)   int16   
  4  SCI           2 ImageHDU       261   (4096, 2048)   float32   
  5  ERR           2 ImageHDU        55   (4096, 2048)   float32   
  6  DQ            2 ImageHDU        47   (4096, 2048)   int16   
  7  D2IMARR       1 ImageHDU        16   (64, 32)   float32   
  8  D2IMARR       2 ImageHDU        16   (64, 32)   float32   
  9  D2IMARR       3 ImageHDU        16   (64, 32)   float32   
 10  D2IMARR       4 ImageHDU        16   (64, 32)   float32   
 11  WCSDVARR      1 ImageHDU        16   (64, 32)   float32   
 12  WCSDVARR      2 ImageHDU        16   (64, 32)   float32   
 13  WCSDVARR      3 ImageHDU      

...

Archived IDC_4bb1536cj-GSC240 in ('SCI', 2)
Writing out a priori WCS IDC_4bb1536cj-GSC240 to headerlet file: jbqz24tzq_flc_9ec245_hlet.fits


updatewcs success.  File info:
Filename: ./jbqz24tzq_flc.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     285   ()      
  1  SCI           1 ImageHDU       263   (4096, 2048)   float32   
  2  ERR           1 ImageHDU        55   (4096, 2048)   float32   
  3  DQ            1 ImageHDU        47   (4096, 2048)   int16   
  4  SCI           2 ImageHDU       261   (4096, 2048)   float32   
  5  ERR           2 ImageHDU        55   (4096, 2048)   float32   
  6  DQ            2 ImageHDU        47   (4096, 2048)   int16   
  7  D2IMARR       1 ImageHDU        16   (64, 32)   float32   
  8  D2IMARR       2 ImageHDU        16   (64, 32)   float32   
  9  D2IMARR       3 ImageHDU        16   (64, 32)   float32   
 10  D2IMARR       4 ImageHDU        16   (64, 32)   float32   
 11  WCSDVARR      1 Im

Processing /home/aswin/hst123/jbqz24u1q_flc.fits['SCI',1]
Updating header for /home/aswin/hst123/jbqz24u1q_flc.fits[1]
Processing /home/aswin/hst123/jbqz24u1q_flc.fits['SCI',2]
Updating header for /home/aswin/hst123/jbqz24u1q_flc.fits[4]
Archived IDC_4bb1536cj-GSC240 in ('SCI', 1)
Archived IDC_4bb1536cj-GSC240 in ('SCI', 2)
Writing out a priori WCS IDC_4bb1536cj-GSC240 to headerlet file: jbqz24u1q_flc_9ec245_hlet.fits




updatewcs success.  File info:
Filename: ./jbqz24u1q_flc.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     285   ()      
  1  SCI           1 ImageHDU       263   (4096, 2048)   float32   
  2  ERR           1 ImageHDU        55   (4096, 2048)   float32   
  3  DQ            1 ImageHDU        47   (4096, 2048)   int16   
  4  SCI           2 ImageHDU       261   (4096, 2048)   float32   
  5  ERR           2 ImageHDU        55   (4096, 2048)   float32   
  6  DQ            2 ImageHDU        47   (4096, 2048)   int16   
  7  D2IMARR       1 ImageHDU        16   (64, 32)   float32   
  8  D2IMARR       2 ImageHDU        16   (64, 32)   float32   
  9  D2IMARR       3 ImageHDU        16   (64, 32)   float32   
 10  D2IMARR       4 ImageHDU        16   (64, 32)   float32   
 11  WCSDVARR      1 ImageHDU        16   (64, 32)   float32   
 12  WCSDVARR      2 ImageHDU        16   (64, 32)   float32   
 13  WCSDVARR      3 ImageHDU      

Processing /home/aswin/hst123/jbqz24u3q_flc.fits['SCI',1]
Updating header for /home/aswin/hst123/jbqz24u3q_flc.fits[1]
Processing /home/aswin/hst123/jbqz24u3q_flc.fits['SCI',2]
Updating header for /home/aswin/hst123/jbqz24u3q_flc.fits[4]
Archived IDC_4bb1536cj-GSC240 in ('SCI', 1)
Archived IDC_4bb1536cj-GSC240 in ('SCI', 2)
Writing out a priori WCS IDC_4bb1536cj-GSC240 to headerlet file: jbqz24u3q_flc_9ec245_hlet.fits




updatewcs success.  File info:
Filename: ./jbqz24u3q_flc.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     285   ()      
  1  SCI           1 ImageHDU       263   (4096, 2048)   float32   
  2  ERR           1 ImageHDU        55   (4096, 2048)   float32   
  3  DQ            1 ImageHDU        47   (4096, 2048)   int16   
  4  SCI           2 ImageHDU       261   (4096, 2048)   float32   
  5  ERR           2 ImageHDU        55   (4096, 2048)   float32   
  6  DQ            2 ImageHDU        47   (4096, 2048)   int16   
  7  D2IMARR       1 ImageHDU        16   (64, 32)   float32   
  8  D2IMARR       2 ImageHDU        16   (64, 32)   float32   
  9  D2IMARR       3 ImageHDU        16   (64, 32)   float32   
 10  D2IMARR       4 ImageHDU        16   (64, 32)   float32   
 11  WCSDVARR      1 ImageHDU        16   (64, 32)   float32   
 12  WCSDVARR      2 ImageHDU        16   (64, 32)   float32   
 13  WCSDVARR      3 ImageHDU      

In [41]:
for image in run_images:
        rawtmp = image.replace('.fits','.rawtmp.fits')
        tmp_images.append(rawtmp)
        
        # Check if rawtmp already exists
        if os.path.exists(rawtmp):
            message = '{file} exists. Skipping...'
            print(message.format(file=rawtmp))
            continue
        
        # Copy the raw data into a temporary file
        shutil.copyfile(image, rawtmp)

        # Clean cosmic rays so they aren't used for alignment
        inst = get_instrument(image).split('_')[0]
        crpars = {'rdnoise': 6.5,
                  'gain': 1.0,
                  'saturate': 70000.0,
                  'sig_clip': 3.0,
                  'sig_frac': 0.1,
                  'obj_lim': 5.0}
        run_cosmic(rawtmp, crpars)

Cleaning cosmic rays in image: ./jbqz22siq_flc.rawtmp.fits
Cleaning cosmic rays in image: ./jbqz22skq_flc.rawtmp.fits
Cleaning cosmic rays in image: ./jbqz22smq_flc.rawtmp.fits
Cleaning cosmic rays in image: ./jbqz22soq_flc.rawtmp.fits
Cleaning cosmic rays in image: ./jbqz23tqq_flc.rawtmp.fits
Cleaning cosmic rays in image: ./jbqz23trq_flc.rawtmp.fits
Cleaning cosmic rays in image: ./jbqz23ttq_flc.rawtmp.fits
Cleaning cosmic rays in image: ./jbqz23tvq_flc.rawtmp.fits
Cleaning cosmic rays in image: ./jbqz24tyq_flc.rawtmp.fits
Cleaning cosmic rays in image: ./jbqz24tzq_flc.rawtmp.fits
Cleaning cosmic rays in image: ./jbqz24u1q_flc.rawtmp.fits
Cleaning cosmic rays in image: ./jbqz24u3q_flc.rawtmp.fits


### Main

In [ ]:
  def run_tweakreg(self, obstable, reference, do_cosmic=True, skip_wcs=False,
#     search_radius=None, update_hdr=True):

#     if self.options['args'].work_dir:
#         outdir = self.options['args'].work_dir
#     else:
#         outdir = '.'

#     os.chdir(outdir)

#     # Get options from object
#     options = self.options['global_defaults']
#     # Check if tweakreg has already been run on each image
#     run_images = self.check_images_for_tweakreg(list(obstable['image']))
#     if not run_images: return('tweakreg success', None)
#     if reference in run_images: run_images.remove(reference)

#     # Records what the offsets are for the files run through tweakreg
#     shift_table = Table([run_images,[np.nan]*len(run_images),
#         [np.nan]*len(run_images)], names=('file','xoffset','yoffset'))

#     # Check if we just removed all of the images
#     if not run_images: #not needed
#         warning = 'WARNING: All images have been run through tweakreg.'
#         print(warning)
#         return(True)

#     print('Need to run tweakreg for images:')
#     self.input_list(obstable['image'], show=True, save=False)

#     tmp_images = []
#     for image in run_images:
#         if self.updatewcs and not skip_wcs:
#             det = '_'.join(self.get_instrument(image).split('_')[:2])
#             wcsoptions = self.options['detector_defaults'][det]
#             self.update_image_wcs(image, wcsoptions)

#         if not do_cosmic:
#             tmp_images.append(image)
#             continue

#         # wfc3_ir doesn't need cosmic clean and assume reference is cleaned
#         if (image == reference or 'wfc3_ir' in self.get_instrument(image)):
#             message = 'Skipping adjustments for {file} as WFC3/IR or reference'
#             print(message.format(file=image))
#             tmp_images.append(image)
#             continue

#         rawtmp = image.replace('.fits','.rawtmp.fits')
#         tmp_images.append(rawtmp)

#         # Check if rawtmp already exists
#         if os.path.exists(rawtmp):
#             message = '{file} exists. Skipping...'
#             print(message.format(file=rawtmp))
#             continue

#         # Copy the raw data into a temporary file
#         shutil.copyfile(image, rawtmp)

#         # Clean cosmic rays so they aren't used for alignment
#         inst = self.get_instrument(image).split('_')[0]
#         crpars = self.options['instrument_defaults'][inst]['crpars']
#         self.run_cosmic(rawtmp, crpars) #nircam l2 is cosmic ray corrected

    modified = False
    ref_images = self.pick_deepest_images(tmp_images)
    deepest = sorted(ref_images, key=lambda im: fits.getval(im, 'EXPTIME'))[-1]
    if (not reference or reference=='dummy.fits'):
        reference = 'dummy.fits'
        message = 'Copying {deep} to reference dummy.fits'
        print(message.format(deep=deepest))
        shutil.copyfile(deepest, reference)
    elif not self.prepare_reference_tweakreg(reference):
        # Can't use this reference image, just use one of the input
        reference = 'dummy.fits'
        message = 'Copying {deep} to reference dummy.fits'
        print(message.format(deep=deepest))
        shutil.copyfile(deepest, reference)
    else:
        modified = True

    message = 'Tweakreg is executing...'
    print(message)

    start_tweak = time.time()

    tweakreg_success = False
    tweak_img = copy.copy(tmp_images)
    ithresh = self.threshold ; rthresh = self.threshold
    shallow_img = []
    thresh_data = None
    tries = 0

    while (not tweakreg_success and tries < 10):
        tweak_img = self.check_images_for_tweakreg(tweak_img)
        if not tweak_img: break
        if tweak_img:
            # Remove images from tweak_img if they are too shallow
            if shallow_img:
                for img in shallow_img:
                    if img in tweak_img:
                        tweak_img.remove(img)

            if len(tweak_img)==0:
                error = 'ERROR: removed all images as shallow'
                print(error)
                tweak_img = copy.copy(tmp_images)
                tweak_img = self.check_images_for_tweakreg(tweak_img)

            # If we've tried multiple runs and there are images in input
            # list with TWEAKSUC and reference image=dummy.fits, we might need
            # to try a different reference image
            success = list(set(tmp_images) ^ set(tweak_img))
            if tries > 1 and reference=='dummy.fits' and len(success)>0:
                # Make random success image new dummy image
                n = len(success)-1
                shutil.copyfile(success[random.randint(0,n)],'dummy.fits')

            # This estimates what the input threshold should be and cuts
            # out images based on number of detected sources from previous
            # rounds of tweakreg
            message = '\n\nReference image: {ref} \n'
            message += 'Images: {im}'
            print(message.format(ref=reference, im=','.join(tweak_img)))

            # Get deepest image and use threshold from that
            deepest = sorted(tweak_img,
                key=lambda im: fits.getval(im, 'EXPTIME'))[-1]

            if not thresh_data or deepest not in thresh_data['file']:
                inp_data = self.get_tweakreg_thresholds(deepest,
                    options['nbright']*4)
                thresh_data = self.add_thresh_data(thresh_data, deepest,
                    inp_data)
            mask = thresh_data['file']==deepest
            inp_thresh = thresh_data[mask][0]
            print('Getting image threshold...')
            new_ithresh = self.get_best_tweakreg_threshold(inp_thresh,
                options['nbright']*4)

            if not thresh_data or reference not in thresh_data['file']:
                inp_data = self.get_tweakreg_thresholds(reference,
                    options['nbright']*4)
                thresh_data = self.add_thresh_data(thresh_data, reference,
                    inp_data)
            mask = thresh_data['file']==reference
            inp_thresh = thresh_data[mask][0]
            print('Getting reference threshold...')
            new_rthresh = self.get_best_tweakreg_threshold(inp_thresh,
                options['nbright']*4)

            if not rthresh: rthresh = self.threshold
            if not ithresh: ithresh = self.threshold

            # Other input options
            nbright = options['nbright']
            minobj = options['minobj']
            search_rad = int(np.round(options['search_rad']))
            if search_radius: search_rad = search_radius

            rconv = 3.5 ; iconv = 3.5 ; tol = 0.25
            if 'wfc3_ir' in self.get_instrument(reference):
                rconv = 2.5
            if all(['wfc3_ir' in self.get_instrument(i)
                for i in tweak_img]):
                iconv = 2.5 ; tol = 0.6
            if 'wfpc2' in self.get_instrument(reference):
                rconv = 2.5
            if all(['wfpc2' in self.get_instrument(i)
                for i in tweak_img]):
                iconv = 2.5 ; tol = 0.5


            # Don't want to keep trying same thing over and over
            if (new_ithresh>=ithresh or new_rthresh>=rthresh) and tries>1:
                # Decrease the threshold and increase tolerance
                message = 'Decreasing threshold and increasing tolerance...'
                print(message)
                ithresh = np.max([new_ithresh*(0.95**tries), 3.0])
                rthresh = np.max([new_rthresh*(0.95**tries), 3.0])
                tol = tol * 1.3**tries
                search_rad = search_rad * 1.2**tries
            else:
                ithresh = new_ithresh
                rthresh = new_rthresh

            if tries > 7:
                minobj = 7

            message = '\nAdjusting thresholds:\n'
            message += 'Reference threshold={rthresh}\n'
            message += 'Image threshold={ithresh}\n'
            message += 'Tolerance={tol}\n'
            message += 'Search radius={rad}\n'
            print(message.format(ithresh='%2.4f'%ithresh,
                rthresh='%2.4f'%rthresh, tol='%2.4f'%tol,
                rad='%2.4f'%search_rad))

            outshifts = os.path.join(outdir, 'drizzle_shifts.txt')

            try:
                tweakreg.TweakReg(files=tweak_img, refimage=reference,
                    verbose=False, interactive=False, clean=True,
                    writecat=True, updatehdr=update_hdr, reusename=True,
                    rfluxunits='counts', minobj=minobj, wcsname='TWEAK',
                    searchrad=search_rad, searchunits='arcseconds', runfile='',
                    tolerance=tol, refnbright=nbright, nbright=nbright,
                    separation=0.5, residplot='No plot', see2dplot=False,
                    fitgeometry='shift',
                    imagefindcfg = {'threshold': ithresh,
                        'conv_width': iconv, 'use_sharp_round': True},
                    refimagefindcfg = {'threshold': rthresh,
                        'conv_width': rconv, 'use_sharp_round': True},
                    shiftfile=True, outshifts=outshifts)

                # Reset shallow_img list
                shallow_img = []

            except AssertionError as e:
                self.tweakreg_error(e)

                message = 'Re-running tweakreg with shallow images removed:'
                print(message)
                for img in tweak_img:
                    nsources = self.get_nsources(img, ithresh)
                    if nsources < 1000:
                        shallow_img.append(img)

            # Occurs when all images fail alignment
            except TypeError as e:
                self.tweakreg_error(e)

            # Record what the shifts are for each of the files run
            message='Reading in shift file: {file}'
            print(message.format(file=outshifts))
            shifts = Table.read(outshifts, format='ascii', names=('file',
                'xoffset','yoffset','rotation1','rotation2','scale1','scale2'))

            self.apply_tweakreg_success(shifts)

            # Add data from output shiftfile to shift_table
            for row in shifts:
                filename = os.path.basename(row['file'])
                filename = filename.replace('.rawtmp.fits','')
                filename = filename.replace('.fits','')

                idx = [i for i,row in enumerate(shift_table)
                    if filename in row['file']]

                if len(idx)==1:
                    shift_table[idx[0]]['xoffset']=row['xoffset']
                    shift_table[idx[0]]['yoffset']=row['yoffset']

            if not self.check_images_for_tweakreg(tmp_images):
                tweakreg_success = True

            tries += 1

    message = 'Tweakreg took {time} seconds to execute.\n\n'
    print(message.format(time = time.time()-start_tweak))

    print(shift_table)

    # tweakreg improperly indexes the CRVAL1 and CRVAL2 values
    # TODO: If drizzlepac fixes this then get rid of this code
    for image in tmp_images:
        rawtmp = image
        rawhdu = fits.open(rawtmp, mode='readonly')

        tweaksuc = False
        if ('TWEAKSUC' in rawhdu[0].header.keys() and
            rawhdu[0].header['TWEAKSUC']==1):
            tweaksuc = True

        if 'wfc3_ir' in self.get_instrument(image): continue

        for i,h in enumerate(rawhdu):
            if (tweaksuc and 'CRVAL1' in h.header.keys() and
                'CRVAL2' in h.header.keys()):
                rawhdu[i].header['CRPIX1']=rawhdu[i].header['CRPIX1']-0.5
                rawhdu[i].header['CRPIX2']=rawhdu[i].header['CRPIX2']-0.5

        rawhdu.writeto(rawtmp, overwrite=True)

    if not skip_wcs:
        for image in run_images:
            # Copy image over now to perform other image header updates
            if (image == reference or 'wfc3_ir' in self.get_instrument(image)):
                continue

            message = '\n\nUpdating image data for image: {im}'
            print(message.format(im=image))
            rawtmp = image.replace('.fits','.rawtmp.fits')

            rawhdu = fits.open(rawtmp, mode='readonly')
            hdu    = fits.open(image, mode='readonly')
            newhdu = fits.HDUList()

            print('Current image info:')
            hdu.info()

            for i, h in enumerate(hdu):
                if h.name=='SCI':
                    if 'flc' in image or 'flt' in image:
                        if len(rawhdu)>=i+2 and rawhdu[i+2].name=='DQ':
                            self.copy_wcs_keys(rawhdu[i], rawhdu[i+2])
                    elif 'c0m' in image:
                        maskfile = image.split('_')[0]+'_c1m.fits'
                        if os.path.exists(maskfile):
                            maskhdu = fits.open(maskfile)
                            self.copy_wcs_keys(rawhdu[i], maskhdu[i])
                            maskhdu.writeto(maskfile, overwrite=True)

                # Skip WCSCORR for WFPC2 as non-standard hdu
                if 'wfpc2' in self.get_instrument(image).lower():
                    if h.name=='WCSCORR':
                        continue

                # Get the index of the corresponding extension in rawhdu.  This
                # can be different from "i" if extensions were added or
                # rearranged
                ver = int(h.ver) ; name = str(h.name).strip()
                idx = -1

                for j,rawh in enumerate(rawhdu):
                    if str(rawh.name).strip()==name and int(rawh.ver)==ver:
                        idx = j

                # If there is no corresponding extension, then continue
                if idx < 0:
                    message = 'Skip extension {i},{ext},{ver} '
                    message += '- no match in {f}'
                    print(message.format(i=i, ext=name, ver=ver, f=rawtmp))
                    continue

                # If we can access the data in both extensions, copy from
                if h.name!='DQ':
                    if 'data' in dir(h) and 'data' in dir(rawhdu[idx]):
                        if (rawhdu[idx].data is not None and
                            h.data is not None):
                            if rawhdu[idx].data.dtype==h.data.dtype:
                                rawhdu[idx].data = h.data

                # Copy the rawtmp extension into the new file
                message = 'Copy extension {i},{ext},{ver}'
                print(message.format(i=idx, ext=name, ver=ver))
                newhdu.append(copy.copy(rawhdu[idx]))

            if 'wfpc2' in self.get_instrument(image).lower():
                # Adjust number of extensions to 4
                newhdu[0].header['NEXTEND']=4

            print('\n\nNew image info:')
            newhdu.info()

            newhdu.writeto(image, output_verify='silentfix', overwrite=True)

            if (os.path.isfile(rawtmp) and not self.options['args'].cleanup):
                os.remove(rawtmp)

    # Clean up temporary files and output
    if os.path.isfile('dummy.fits'):
        os.remove('dummy.fits')

    if not self.options['args'].keep_objfile:
        for file in glob.glob('*.coo'):
            os.remove(file)

    if modified:
        # Re-sanitize reference using main sanitize function
        self.sanitize_reference(reference)

    return(tweakreg_success, shift_table)